In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
import csv
from bs4 import BeautifulSoup

In [ ]:
# setting the headers for the CSV file
headers = ['title', 'type', 'address']  # default

# dynamic variable for file name for saving the HTML markup text to save
fileName = 'html-markup.txt'

# writing the headers to the CSV file
with open('data.csv', 'w', encoding='utf-8', newline='') as f:

    csv_writer = csv.DictWriter(f, fieldnames=headers)

    csv_writer.writeheader()

# this function basically reads the file data and uses its markup data for the beautifulSoup as an argument
def scrape_data(page):
    with open(fileName, 'r', encoding='utf-8') as f:
        contents = f.read()

    doc = BeautifulSoup(contents, features='html5lib')  # this doc holds the markup data read from the file

    titles = [k.text for k in doc.find_all('span', class_='_hc69qa')]

    types = [k.text for k in doc.find_all('span', class_='_oqoid')]

    addresses = [k.text for k in doc.find_all('span', class_='_tluih8')]

    # csv data row writing
    with open('data.csv', 'a', encoding='utf-8', newline='') as f:
        writer = csv.writer(f)

        for m in range(1, len(titles)):
            writer.writerow([titles[m], types[m], addresses[m]])

    print(f'\nfinished parsing {page}')  # gives the status for each page if the parsing is done

browser = webdriver.Chrome()  # setting the webdriver for chrome

# maximazing allows the parser configurations to work more smoothly for avoiding 'no element exceptions'
browser.maximize_window()

# set the url you want to your own 2gis url for which you want to get the data
url = 'https://2gis.ru/kazan/search/%D1%81%D1%82%D1%80%D0%BE%D0%B8%D1%82%D0%B5%D0%BB%D1%8C%D0%BD%D1%8B%D0%B5%20%D0%BA%D0%BE%D0%BC%D0%BF%D0%B0%D0%BD%D0%B8%D0%B8/rubricId/298?m=49.184037%2C55.784963%2F11.47'

browser.get(url)  # opening the url via webdriver

# implicitly waiting for the page to load its contents in seconds
browser.implicitly_wait(10)  # customize the time if your internet is slow

page_element = browser.find_element(
    By.XPATH, "(//span[@class='_18lf326a'])[1]")

num_of_pages = (int(page_element.text)//12)+3  # calculates the number of pages to click

# main loop, it works on page limitition of the corresponding data search passed to the webdriver
for page in range(1, num_of_pages):
    with open(fileName, 'w', encoding='utf-8') as f:
        f.write(browser.page_source)

    scrape_data(page)

    time.sleep(1.8)

    # getting the scroll element in DOM for getting the whole HTML markup so that beautifulSoup can parse it according to the headers list config
    scroll_container = browser.find_element(
        By.XPATH, "(//div[@class='_15gu4wr'])[3]")

    # scrolling the <ul></ul> element
    browser.execute_script("arguments[0].scrollIntoView(false);", scroll_container)

    # clicking on the next page DOM element
    browser.find_element(By.XPATH, "//div[@class='_5ocwns']//div[2]").click()

